In [1]:
import numpy as np
from qecdec import RotatedSurfaceCode_Memory
from qecdec import BPDecoder, DMemBPDecoder
from qecdec.utils import visualize_bp_decoding_process

In [2]:
d = 5
p = 0.005

expmt = RotatedSurfaceCode_Memory(
    d=d, 
    rounds=3,
    basis='Z', 
    data_qubit_error_rate=p, 
    meas_error_rate=p,
)

bp_decoder = BPDecoder(pcm=expmt.chkmat, prior=expmt.prior, max_iter=50)
np.random.seed(42)
gamma = np.random.uniform(-0.25, 0.95, expmt.prior.shape)
dmem_bp_decoder = DMemBPDecoder(pcm=expmt.chkmat, prior=expmt.prior, max_iter=50, gamma=gamma)


In [3]:
num_shots = 1000

sampler = expmt.circuit.compile_detector_sampler(seed=0)
syndrome_batch, observable_batch = sampler.sample(num_shots, separate_observables=True)
syndrome_batch = syndrome_batch.astype(np.int8)
observable_batch = observable_batch.astype(np.int8)

In [4]:
ehat_batch = bp_decoder.decode_batch(syndrome_batch)

unmatched_syndrome_mask = np.any(syndrome_batch != (ehat_batch @ expmt.chkmat.T) % 2, axis=1)

np.where(unmatched_syndrome_mask)[0]

array([164, 378, 581, 745, 755, 819, 840])

In [5]:
shot_idx = 819

syndrome = syndrome_batch[shot_idx]
observable = observable_batch[shot_idx]

In [6]:
ehat = bp_decoder.decode(syndrome, record_llr_history=True)
llr_history = bp_decoder.get_llr_history()

print("Number of iterations: ", llr_history.shape[0])
print("Has the decoder matched the syndrome? ", np.all(syndrome == (expmt.chkmat @ ehat) % 2))
print("Has the decoder correctly predicted the logical observable? ", np.all(observable == (expmt.obsmat @ ehat) % 2))

visualize_bp_decoding_process(expmt.chkmat, expmt.detector_coords, expmt.error_coords, syndrome, llr_history, zaxis_stretch=4)

Number of iterations:  50
Has the decoder matched the syndrome?  False
Has the decoder correctly predicted the logical observable?  True


In [7]:
ehat = dmem_bp_decoder.decode(syndrome, record_llr_history=True)
llr_history = dmem_bp_decoder.get_llr_history()

print("Number of iterations: ", llr_history.shape[0])
print("Has the decoder matched the syndrome? ", np.all(syndrome == (expmt.chkmat @ ehat) % 2))
print("Has the decoder correctly predicted the logical observable? ", np.all(observable == (expmt.obsmat @ ehat) % 2))

visualize_bp_decoding_process(expmt.chkmat, expmt.detector_coords, expmt.error_coords, syndrome, llr_history, zaxis_stretch=4)

Number of iterations:  9
Has the decoder matched the syndrome?  True
Has the decoder correctly predicted the logical observable?  True
